In [56]:
import tweepy
import csv
from textblob import TextBlob
import sys
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from time import sleep

In [57]:
# Get tokens
def getTokens():
    credentials =[]
    with open("../login.csv", 'r') as file:
        csvreader = csv.reader(file)
        for row in csvreader:
            credentials.append(row)
    # Don't return headers
    return credentials[1]

In [58]:
def percentage(part,whole):
     return 100 * float(part)/float(whole)

In [59]:
tokens = getTokens()
consumerKey = tokens[0]
consumerSecret = tokens[1]
accessToken = tokens[2]
accessTokenSecret = tokens[3]

In [60]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [61]:
#Sentiment Analysis
# keyword = input("Please enter keyword or hashtag to search: ")
keyword = "bitcoin"
# noOfTweet = int(input("Please enter how many tweets to analyze: "))

In [62]:
tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(100)
positive:int = 0
negative:int = 0
neutral:int = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []

In [ ]:

def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: remove_stopwords(x))

In [63]:
#Removing stopwords
stopwords = nltk.corpus.stopwords.words('english')

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    print(temp)
    sleep(5)
    return temp



In [64]:
for tweet in tweets:
    tweet_list.append(clean_tweet(tweet.text))
    analysis = TextBlob(tweet.text)
    score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    polarity += analysis.sentiment.polarity
    print(f'pos = {pos}, neg = {neg}')
    if  pos < neg:
        negative_list.append(tweet.text)
        negative += 1
    elif pos > neg:
        positive_list.append(tweet.text)
        positive += 1
    elif pos == neg:
        neutral_list.append(tweet.text)
        neutral += 1

C:\Users\jerem\AppData\Local\Temp\ipykernel_5496\1563610548.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(tweet) == np.float:


rt 2021 vs 2022 drawdown slightly smaller longer period looks like bottom forming
pos = 0.153, neg = 0.0
elon u better talking bitcoin prices
pos = 0.293, neg = 0.0
rt want make big difference earning money join
pos = 0.163, neg = 0.0
rt always enough research
pos = 0.0, neg = 0.0
rt fuck putin fuck zelensky fuck biden fuck nato fuck un nation states buy
pos = 0.0, neg = 0.593
rt showing power decentralized money forbes
pos = 0.0, neg = 0.0
rt exploring investor perceptions inflation hedge mornings first mover americas reports htt
pos = 0.0, neg = 0.0
rt typical ct like ok bitcoin didnt pump first 2 minutes invasion country wa
pos = 0.118, neg = 0.0
rt still perfectly track
pos = 0.318, neg = 0.0
rt crypto miner could raise bitcoin mining hashrate 95 new intel deal company
pos = 0.0, neg = 0.0
gay trans propaganda big infertility op imo
pos = 0.0, neg = 0.154
rt q8 trade
pos = 0.0, neg = 0.0
rt taikun global token coming binance smart chain
pos = 0.13, neg = 0.0
one best project team i

In [ ]:
positive = percentage(positive, 100)
negative = percentage(negative, 100)
neutral = percentage(neutral, 100)
polarity = percentage(polarity, 100)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

In [ ]:
#Number of Tweets (Total, Positive, Negative, Neutral)
tweet_list = pd.DataFrame(tweet_list)
neutral_list = pd.DataFrame(neutral_list)
negative_list = pd.DataFrame(negative_list)
positive_list = pd.DataFrame(positive_list)
print(f"total number: {len(tweet_list)}")
print(f"positive number: {len(positive_list)}")
print(f"negative number: {len(negative_list)}")
print(f"neutral number: {len(neutral_list)}")

In [ ]:
tweet_list.drop_duplicates(inplace = True)
#Creating PieCart
labels = ['Positive ['+str(positive)+'%]' , 'Neutral ['+str(neutral)+'%]','Negative ['+str(negative)+'%]']
sizes = [positive, neutral, negative]
colors = ['yellowgreen', 'blue','red']
patches, texts = plt.pie(sizes,colors=colors, startangle=90)
plt.style.use('default')
plt.legend(labels)
plt.title("Sentiment Analysis Result for keyword= "+keyword+"" )
plt.axis('equal')
plt.show()

In [ ]:
#Cleaning Text (RT, Punctuation etc)
#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0–9]+)|(#[0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)

tw_list['text']


0      buna ne kadar etkileşim rt fav gelirse o kada...
1      rûpela me ya   nûjen bû! wek guleke zer xwe b...
2      _nft more then 2000$ in nft to gave away foll...
3      ／\n6カ月連続キャンペーン第2弾🎊\n🎉#フォローrtキャンペーン 🎉\n1,000円分...
4                       itcoin no or never\n👇🏾👇🏾👇🏾\n   
                            ...                        
95     today we scalp longed $hbar\n\nentry: $0.2056...
96     presale is live now, buy tokens for lowest pr...
97                     _tr      _ai  _o_l_o_        …  
98     diamondhand\n     era shibnobi , great projec...
99     in a gun fight, nobody wants to own the secon...
Name: text, Length: 82, dtype: object

In [ ]:
#Calculating Negative, Positive, Neutral and Compound values
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
        tw_list.loc[index, 'neg'] = neg
        tw_list.loc[index, 'neu'] = neu
        tw_list.loc[index, 'pos'] = pos
        tw_list.loc[index, 'compound'] = comp
tw_list.head(10)

In [ ]:
#Creating new data frames for all sentiments (positive, negative and neutral)
tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [ ]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

In [ ]:
# create data for Pie Chart
pc = count_values_in_column(tw_list,"sentiment")
names= pc.index
size=pc["Percentage"]
 
# Create a circle for the center of the plot
my_circle=plt.Circle( (0,0), 0.7, color='white')
plt.pie(size, labels=names, colors=['green','blue','red'])
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.show()

In [ ]:
#Function to Create Wordcloud
def create_wordcloud(text):
    mask = np.array(Image.open("cloud.png"))
    stopwords = set(STOPWORDS)
    wc = WordCloud(background_color="white",
    mask = mask,
    max_words=3000,
    stopwords=stopwords,
    repeat=True).generate(str(text)).to_file("wc.png")
    # wc.generate(str(text))
    # wc.to_file("wc.png")
    print("Word Cloud Saved Successfully")
    path="wc.png"
    display(Image.open(path))

In [ ]:
#Creating wordcloud for all tweets
create_wordcloud(tw_list["text"].values)

In [ ]:
#Creating wordcloud for positive sentiment
create_wordcloud(tw_list_positive["text"].values)

In [ ]:
#Creating wordcloud for negative sentiment
create_wordcloud(tw_list_negative["text"].values)

In [ ]:
#Calculating tweet's lenght and word count
tw_list['text_len'] = tw_list['text'].astype(str).apply(len)
tw_list['text_word_count'] = tw_list['text'].apply(lambda x: len(str(x).split()))
round(pd.DataFrame(tw_list.groupby("sentiment").text_len.mean()),2)

tw_list['text']

In [ ]:
round(pd.DataFrame(tw_list.groupby("sentiment").text_word_count.mean()),2)

In [ ]:
#Removing Punctuation
def remove_punct(text):
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0–9]+', '', text)
    return text

tw_list['punct'] = tw_list['text'].apply(lambda x: remove_punct(x))

tw_list['text']

In [ ]:
#Appliyng tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text
tw_list['tokenized'] = tw_list['punct'].apply(lambda x: tokenization(x.lower()))


KeyError: 'punct'

In [ ]:
#Removing stopwords
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: remove_stopwords(x))

In [ ]:
#Appliyng Stemmer
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text
tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: stemming(x))

In [ ]:
#Cleaning Text
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
tw_list.head()